In [1]:
import pandas as pd
import os

In [2]:
data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/raw_data/'
cleaned_data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/cleaned_data/'
variables_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/variables/'

In [3]:
xl = pd.ExcelFile(data_path+'Data_as_per_requested_till_31st_Aug_2023.xlsx')
res = len(xl.sheet_names) #Number of sheets

df = pd.read_excel(data_path+'Data_as_per_requested_till_31st_Aug_2023.xlsx',
                   sheet_name=0)

In [4]:
cols = df.columns.to_list()

In [5]:
cols_to_remove_for_aggregation = ['Id','Report_Date','Report Date',
                                  'District_Id','District Id',
                                  'Revenue_Name','Revenue Circle Name',
                                  'RC_Id', 'Revenue Circle Id', 
                                  'District_Name', 'District Name',
                                  'DFC_Id','DCF_Id',
                                  'Block_Name',
                                  'Type', 'Name','Location',''
                                 'Relief_Distribution_Center_Name', 'Child_friendly_Space',
                                 'Latitude', 'Longitude',
                                  'Relief_Distribution_Other_Item_Name', 'Relief_Distribution_Other_Item_Qty',
                                 'Relief_Distribution_Other_Item_Unit',
                                 'Human_Live_Lost_Type']

for col_to_remove in cols_to_remove_for_aggregation:
    try:
        cols.remove(col_to_remove)
    except:
        continue

In [6]:
cols

['Population_affected_Male',
 'Population_affected_Female',
 'Population_affected_Total',
 'House_Fully_Damaged_Kuccha',
 'House_Fully_Damaged_Pukka',
 'Total_House_Fully_Damaged',
 'House_Partially_Damaged_Kuccha',
 'House_Partially_Damaged_Pukka',
 'Total_House_Partially_Damaged',
 'House_Damaged_Others_Huts',
 'House_Damaged_Others_CattleShed',
 'Total_House_Damaged_Others',
 'Animal_Affected_Big',
 'Animal_Affected_Small',
 'Animal_Affected_Poultry',
 'Total_Animal_Affected',
 'Animal_Washed_Away_Big',
 'Animal_Washed_Away_Small',
 'Animal_Washed_Away_Poultry',
 'Total_Animal_Washed_Away',
 'Relief_Distribution_Rice_In_Q',
 'Relief_Distribution_Dal_In_Q',
 'Relief_Distribution_Salt_In_Q',
 'Relief_Distribution_Oil_In_Q',
 'Relief_Distribution_Cattel_Feed_Green_Fooder_In_Q',
 'Relief_Distribution_Cattel_Feed_Wheat_Bran_In_Q',
 'Relief_Distribution_Cattel_Feed_Rice_Bran_In_Q',
 'Relief_Distribution_Liquid_BabyFood_In_Q',
 'Relief_Distribution_Liquid_BabyFood_In_Unit',
 'Male_Camp',
 

In [7]:
grouped_df = df.groupby(['Report_Date', 'RC_Id'])[cols].sum()
grouped_df

Population_affected_Male  Population_affected_Female  \
Report_Date RC_Id                                                         
2021-05-15  2.0                         0.0                         0.0   
            3.0                         0.0                         0.0   
            4.0                         0.0                         0.0   
            10.0                        0.0                         0.0   
            16.0                        0.0                         0.0   
...                                     ...                         ...   
2023-08-30  175.0                       0.0                         0.0   
            176.0                       0.0                         0.0   
            177.0                       0.0                         0.0   
            178.0                       0.0                         0.0   
            188.0                    1127.0                      1095.0   

                   Population_affected_Total  House_Fully_Damaged_Kuccha  \
Report_Date RC_Id                                                          
2021-05-15  2.0                          0.0                         0.0   
            3.0                          0.0                         0.0   
            4.0                          0.0                         0.0   
            10.0                         0.0                         0.0   
            16.0                         0.0                         0.0   
...                                      ...                         ...   
2023-08-30  175.0                        0.0                         0.0   
            176.0                        0.0                         0.0   
            177.0                        0.0                         0.0   
            178.0                        0.0                         0.0   
            188.0                     2901.0                         0.0   

                   House_Fully_Damaged_Pukka  Total_House_Fully_Damaged  \
Report_Date RC_Id                                                         
2021-05-15  2.0                          0.0                        0.0   
            3.0                          0.0                        0.0   
            4.0                          0.0                        0.0   
            10.0                         0.0                        0.0   
            16.0                         0.0                        0.0   
...                                      ...                        ...   
2023-08-30  175.0                        0.0                        0.0   
            176.0                        0.0                        0.0   
            177.0                        0.0                        0.0   
            178.0                        0.0                        0.0   
            188.0                        0.0                        0.0   

                   House_Partially_Damaged_Kuccha  \
Report_Date RC_Id                                   
2021-05-15  2.0                               0.0   
            3.0                               0.0   
            4.0                               0.0   
            10.0                              0.0   
            16.0                              0.0   
...                                           ...   
2023-08-30  175.0                             0.0   
            176.0                             0.0   
            177.0                             0.0   
            178.0                             0.0   
            188.0                             0.0   

                   House_Partially_Damaged_Pukka  \
Report_Date RC_Id                                  
2021-05-15  2.0                              0.0   
            3.0                              0.0   
            4.0                              0.0   
            10.0                             0.0   
            16.0                             0.0   
...                     

In [8]:
variables = grouped_df.columns

In [17]:
frims_rc_id_mapping = pd.read_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/frims_rc_id_mapping.csv')
frims_rc_id_mapping = frims_rc_id_mapping[['object_id','RC_Id']]

In [18]:
unmapped_rcs = []

for variable in variables:
    variable_df = grouped_df[variable].reset_index()
    variable_df['year'] = variable_df.Report_Date.dt.year
    variable_df['month'] = variable_df.Report_Date.dt.month
    variable_df['month'] = variable_df.month.map("{:02}".format)
    variable_df = variable_df.groupby(['year','month','RC_Id'])[variable].sum().reset_index()
    variable_df['year_month'] = variable_df['year'].astype(str) + '_' + variable_df['month'].astype(str)
    
    for year_month in variable_df['year_month'].unique():
        variable_df_monthly = variable_df[variable_df['year_month']==year_month]
        variable_df_monthly = variable_df_monthly.merge(frims_rc_id_mapping, on='RC_Id', how='left')
        
        #variable_df_monthly = variable_df_monthly[['object_id', variable]]
        
        issue = variable_df_monthly.object_id.isnull().sum()
        if issue>0:
            unmapped_rcs.append(variable_df_monthly[variable_df_monthly.object_id.isnull()]['RC_Id'].to_list())
            
        if os.path.exists(variables_path+variable):
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)
        else:
            os.mkdir(variables_path+variable)
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)

In [19]:
def flatten(l):
    return [item for sublist in l for item in sublist]

set(flatten(unmapped_rcs))

set()

In [16]:
df[df['RC_Id']==159]

,Report_Date,District_Id,District_Name,DFC_Id,RC_Id,Revenue_Name,Population_affected_Male,Population_affected_Female,Population_affected_Total,House_Fully_Damaged_Kuccha,...,Relief_Distribution_Liquid_BabyFood_In_Unit,Male_Camp,Female_Camp,Children_Camp,Rescue_Operation_Boats_Deployed,Rescue_Operation_Person_Evacuated_by_Boats,Rescue_Operation_Animal_Evacuated_by_Boats,Rescue_Operation_Helicopters_Deployed,Rescue_Operation_Helicopter_Person_Evacuated,Rescue_Operation_Medical_Team_Deployed
65,2021-05-15,13,Sonitpur,65,159.0,Thelamara,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,2021-05-16,13,Sonitpur,222,159.0,Thelamara,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
380,2021-05-17,13,Sonitpur,387,159.0,Thelamara,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,2021-05-18,13,Sonitpur,587,159.0,Thelamara,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
704,2021-05-19,13,Sonitpur,720,159.0,Thelamara,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69401,2023-08-26,13,Sonitpur,83847,159.0,Thelamara,NaN,NaN,NaN,0.0,...,1.0,0.0,0.0,0.0,2.0,44.0,0.0,0.0,0.0,0.0
69562,2023-08-27,13,Sonitpur,84014,159.0,Thelamara,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69594,2023-08-28,13,Sonitpur,84043,159.0,Thelamara,NaN,NaN,NaN,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69808,2023-08-29,13,Sonitpur,84247,159.0,Thelamara,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# FRIMS RC ID MAPPING

In [177]:
import geopandas as gpd
rc_gdf = gpd.read_file('/home/krishna/IDS-DRR-Data-Pipeline/Maps/Assam_Revenue_Circles/assam_revenue_circle_nov2022.geojson')

In [178]:
rc_gdf = rc_gdf[['revenue_ci', 'object_id','district_3']]

In [121]:
frims_rcs = df[['Revenue_Name','RC_Id','District_Name']].drop_duplicates()
frims_rcs

,Revenue_Name,RC_Id,District_Name
0,Jorhat West,92.0,Jorhat
1,Barkhetri,145.0,Nalbari
2,Goroimari,105.0,Kamrup
3,Sadiya,165.0,Tinsukia
4,Doboka,86.0,Hojai
...,...,...,...
16109,NaN,NaN,Nagaon
16505,NaN,NaN,Kokrajhar
30182,NaN,NaN,Cachar
35914,NaN,NaN,Karimganj


In [122]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [123]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [124]:
fuzzymatch = fuzzy_merge(rc_gdf, frims_rcs, 'revenue_ci', 'Revenue_Name', threshold=80,limit=1)
fuzzymatch

/tmp/ipykernel_7127/339486232.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
/tmp/ipykernel_7127/339486232.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


,revenue_ci,object_id,district_3,matches
0,Gossaigaon (Pt),101,KOKRAJHAR,Gossaigaon
1,Bhowraguri,102,KOKRAJHAR,Bhawraguri
2,Dotoma,103,KOKRAJHAR,Dotma
3,Kokrajhar (Pt),104,KOKRAJHAR,Kokrajhar
4,Bagribari (Pt),105,KOKRAJHAR,Bagribari
...,...,...,...,...
175,Sapekhati,278,CHARAIDEO,Sapekhati
176,Sonari,279,CHARAIDEO,Sonari
177,Ujani Majuli,280,MAJULI,Ujani Majuli
178,Majuli,281,MAJULI,Majuli


In [125]:
fuzzymatch.merge(frims_rcs, left_on='matches', right_on='Revenue_Name').to_csv('frims_rc_id_mapping.csv', index=False)